[Look Up](https://www.luogu.org/problemnew/show/P2947)。给定一数组，求各数字右边第一个比该数字大的数，没有则设置0.

思路：单调栈的典型应用。从后往前扫描数组，设立一个栈，栈中始终保存比当前数字大的元素，若有小的全部弹出。

In [ ]:
def LookUp(nums):
    n=len(nums)
    res=[0]*n
    s=list()
    
    for idx in range(n-1,-1,-1):
        # 先排空栈中不大于当前数的所有数字
        while s and nums[s[-1]]<=nums[idx]:
            s.pop()
        
        # 栈中剩下的元素都是比当前大的数了
        if not s:    # 栈为空
            res[idx]=0
        else:    # 栈不空，比当前大的最近的数就是栈顶元素
            res[idx]=s[-1]
        
        s.append(idx)
        
    return res

[Remove K Digits](https://leetcode.com/problems/remove-k-digits/)。给一个数字字串，删掉其中的$k$位，使得得到的数字最小。

思路：单调栈。所有位顺序入栈，当某一位小于栈顶元素时，需要弹栈。注意弹栈的次数不能超过$k$次。不难，实现逻辑稍微复杂一点。

In [ ]:
def removeKdigits(num: str, k: int) -> str:
    n = len(num)
    if k >= n:
        return '0'

    s = list()
    cnt = 0

    for ch in num:
        if cnt < k:
            if not s:
                s.append(ch)
                continue

            while s and int(ch) < int(s[-1]) and cnt < k:
                s.pop()
                cnt += 1
            s.append(ch)

        else:
            s.append(ch)

    return str(int(''.join(s[:n-k])))